<div align="center">

# **Limpieza**

</div>

## Librerias

In [218]:
library(dplyr)
library(tidyr)
source("../src/data/dividir_polizas_por_anio.R")
source("../src/data/ajustar_inflacion.R")
source("../src/data/Freedman_Diaconis.R")

## Data

In [219]:
df_input <- read.csv("../data/processed/datos_agrupados.csv")

In [220]:
colSums(is.na(df_input))

X          Amparo     SumaDePagos          Modelo           Color 
              0            4320            4320               0               0 
     Carroceria           MARCA     Referencia1     Referencia2 CLASE_FASECOLDA 
              0               0               0               0               0 
  TIPO_VEHICULO        SERVICIO       Sexo_Aseg            Edad           Desde 
              0               0             625               0               0 
          Hasta    Vr_Comercial            Pago 
              0               0               0

## Preparacion de los datos

Nombres de variables

In [221]:
df_input$Amparo <- gsub("DA�OS", "DAÑOS", df_input$Amparo)

Nulos en Amp

In [222]:
df_input <- df_input %>%
  mutate(
    # Convertir NA a "No aplica"
    Amparo = ifelse(is.na(Amparo), "No aplica", Amparo)
    )

Nulos en SumaDePagos

In [223]:
head(sort(unique(df_input$SumaDePagos)))

[1]     0 26248 60000 69257 70000 80000

In [224]:
pago_minimo <- 100000
df_input <- df_input %>% mutate(
    SumaDePagos = case_when(
      is.na(SumaDePagos) ~ 0,
      SumaDePagos < pago_minimo ~ 0,
      TRUE ~ SumaDePagos)
)

df_input <- df_input %>% mutate(
  Accidentado = ifelse(SumaDePagos >= pago_minimo, 1, 0)
)

El pago minimo se escogio como el valor de un espejo en 2015

Nulos en genero

In [225]:
df_input <- df_input %>% mutate(
    Sexo_Aseg = ifelse(is.na(Sexo_Aseg), "No_Binarie", Sexo_Aseg)
)

In [226]:
nrow(df_input[(df_input$Sexo_Aseg == "No_Binarie"), ])/nrow(df_input)

[1] 0.125

Segmentando por año

In [227]:
df_input <- dividir_polizas_por_anio(df_input)

Calculo de exposicion

In [228]:
df_input$exposicion <- as.numeric(df_input$Hasta - df_input$Desde) + 1

Ajustando a inflacion de 2015

In [229]:
max(df_input$Hasta)

[1] "2015-02-21"

In [230]:
df_input <- ajustar_inflacion_2015(df_input)

Llevamos todo a los mismo valores presentes ajustando la inflacion.

Ajustando valor comercial

In [231]:
vr_minimo <- 4000000
df_input <- df_input[(df_input$Vr_Comercial > vr_minimo), ]

El valor comercial minimo se escogio como el valor de un carro hyundai Excel en 2015

In [232]:
colSums(is.na(df_input))

X          Amparo     SumaDePagos          Modelo           Color 
              0               0               0               0               0 
     Carroceria           MARCA     Referencia1     Referencia2 CLASE_FASECOLDA 
              0               0               0               0               0 
  TIPO_VEHICULO        SERVICIO       Sexo_Aseg            Edad           Desde 
              0               0               0               0               0 
          Hasta    Vr_Comercial            Pago     Accidentado      exposicion 
              0               0               0               0               0

In [233]:
summary(df_input)

       X           Amparo           SumaDePagos          Modelo         
 Min.   :   1   Length:9190        Min.   :       0   Length:9190       
 1st Qu.:1244   Class :character   1st Qu.:       0   Class :character  
 Median :2491   Mode  :character   Median :       0   Mode  :character  
 Mean   :2494                      Mean   :  272939                     
 3rd Qu.:3732                      3rd Qu.:       0                     
 Max.   :5000                      Max.   :35206209                     
    Color            Carroceria           MARCA           Referencia1       
 Length:9190        Length:9190        Length:9190        Length:9190       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                           

## Creación de grupos de edad

Exportando los resultados

In [234]:
write.csv(df_input, "../data/processed/datos_limpios.csv", row.names = FALSE)